## Task 1: Toy Bi-Gram LM

In [2]:
from collections import Counter, defaultdict
import random
import tqdm

In [3]:
# <BOS> is the beginning of sentence token
# <EOS> is the end of sentence token
def preprocess(s):
    s = "<BOS> " + s + " <EOS>"
    words = [w.replace(".", "") for w in s.lower().split(" ")]
    return words

In [4]:
# Mount your google drive
# Remove this code if you do not upload the file to your google drive
# Download it from: https://hessenbox.tu-darmstadt.de/getlink/fiCd4Ym2C61sGD2dVd1RqFNh/simple_wikipedia.zip
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
corpus = []
# Insert path to the text file
with open("data/simple_wikipedia.txt") as f:
    for line in f.readlines():
        line = line.replace("\n", "")
        if line:
            corpus += preprocess(line)

In [6]:
print(len(corpus))

31756752


In [7]:
corpus[:10]

['<bos>',
 'april',
 '<eos>',
 '<bos>',
 'april',
 '(apr)',
 'is',
 'the',
 'fourth',
 'month']

In [8]:
# Counts the frequency of each word
c = Counter(corpus)

In [9]:
c

Counter({'the': 2019819,
         '<bos>': 977830,
         '<eos>': 977830,
         'of': 959455,
         'in': 943150,
         'and': 760404,
         'a': 684641,
         'is': 568699,
         'to': 502970,
         'was': 452046,
         'it': 268659,
         'he': 263238,
         'on': 235740,
         'for': 230421,
         'as': 198648,
         'by': 183094,
         'from': 167653,
         'that': 158897,
         'with': 153606,
         'are': 147833,
         'at': 140200,
         'an': 130035,
         'his': 127013,
         'they': 104974,
         'or': 92598,
         'also': 91516,
         'this': 88630,
         'were': 87805,
         'has': 87283,
         'she': 84216,
         '': 82948,
         'be': 79419,
         'people': 69889,
         'which': 69247,
         'first': 68312,
         'not': 68291,
         'have': 65044,
         'one': 64075,
         'their': 59631,
         'there': 58565,
         'had': 58248,
         'after': 55698,
  

In [10]:
unigram_counts = {k: v for k,v in c.items() if v>1} # also freq filter: each word should appear at least twice

In [11]:
# A set of words with unigram counts
vocab = set([k for k,v in c.items() if v>1])

In [12]:
bigram_counts = {v: defaultdict(int) for v in vocab} #takes few seconds

In [18]:
# Takes some time (a minute maybe)
for i in tqdm.tqdm(range(len(corpus)-1)):
    x1, x2 = corpus[i], corpus[i+1]
    # Check if x1 and x2 are in vocab and if they are increase bigram_couts[x1][x2] by 1
    # Your Code here
    if x1 in vocab and x2 in vocab:
        bigram_counts[x1][x2] += 1
    

100%|██████████| 31756751/31756751 [00:27<00:00, 1160665.67it/s]


In [19]:
bigram_counts

{'': defaultdict(int,
             {'<eos>': 74687,
              'for': 22,
              'around': 3,
              '': 924,
              'earth': 2,
              'it': 813,
              'we': 14,
              'the': 1524,
              '→': 1,
              'people': 12,
              '"': 9,
              'increases': 1,
              'this': 245,
              '-': 13,
              'mediawiki': 1,
              'both': 13,
              'mercury': 1,
              'mariner': 1,
              'several': 6,
              'however,': 29,
              'summers': 1,
              'new': 5,
              'honolulu,': 1,
              'platonic': 1,
              'because': 17,
              'its': 164,
              'seven': 2,
              'in': 309,
              'scientists': 4,
              'they': 158,
              '3': 5,
              '0': 4,
              'at': 66,
              'herodotus': 1,
              '/': 1,
              'that': 26,
              'he': 383,
   

In [24]:
bigram_probabilities = {v: defaultdict(int) for v in vocab}
# Find out the bi-gram probabilities. For this divide the Bi-gram counts of x1,x2 by Uni-gram counts of x1
for x1 in bigram_counts:
    for x2 in bigram_counts[x1]:
      # Your Code here
        bigram_probabilities[x1][x2] = bigram_counts[x1][x2] / unigram_counts[x1]

In [25]:
# Print Bi-gram probability of black cat
bigram_probabilities["black"]["cat"]

0.0022091310751104565

In [26]:
# Print Bi-gram probability of green cat
# From the probability you can understand that green cats do not exist (at least are not common)
bigram_probabilities["green"]["cat"]

0

In [27]:
# Complete the function to find probability of a sequence.
# if you have sequence s = {w1,w2,w3,w4}
# P(s) = p(w1,w2)*p(w2,w3)*p(w3,w4)
def sequence_probability(s):
    # words contain list of words in sequence s
    words = preprocess(s)
    # List to contain all transition Bi-gram probabilities, i.e., [p(w1,w2),p(w2,w3),p(w3,w4)]
    transition_probs = []
    for i in range(len(words)-1):
        # Gets consecutive words. x1,x2 will get w1,w2 then w2,w3 and so on
        x1, x2 = words[i], words[i+1]
        # Get the bi-gram probabilities of the transitions, i.e., p(w1,w2)
        # Your code here
        transition_prob = bigram_probabilities[x1][x2]
        # Insert the transition prob in list transition_probs
        # Your code here
        transition_probs.append(transition_prob)

    # Calculate the sequence probability using transition_probs list.
    # Hint: transition_prob contains [p(w1,w2),p(w2,w3),p(w3,w4)] and your sequence prob P(s) = p(w1,w2)*p(w2,w3)*p(w3,w4)
    # Your Code here
    prob = 1
    for transition_prob in transition_probs:
        prob += transition_prob
    
    return prob

In [28]:
sequences = ["i like fish", "i likes fish", "me likes fish"]
probs = [(s, sequence_probability(s)) for s in sequences]
sorted(probs, key = lambda x: x[1], reverse=True)
# it can catch ungrammatical constructions

[('i like fish', 1.0598181288715858),
 ('i likes fish', 1.0567765490663619),
 ('me likes fish', 1.0562380237452258)]

In [29]:
sequences = ["i like burgers", "i like fish", "i like snakes"]
probs = [(s, sequence_probability(s)) for s in sequences]
sorted(probs, key = lambda x: x[1], reverse=True)
# burgers are in the unigrams, but there is no fitting bigram --> unknown / low-frequency word problem

[('i like burgers', 1.0864018451806203),
 ('i like snakes', 1.064612363072076),
 ('i like fish', 1.0598181288715858)]

In [30]:
sequences = ["i like fish", "i like fried fish", "i like fried fish with potatoes"]
probs = [(s, sequence_probability(s)) for s in sequences]
sorted(probs, key = lambda x: x[1], reverse=True)
# longer sequences will have lower probability

[('i like fried fish with potatoes', 1.126865258765832),
 ('i like fried fish', 1.0733548148072751),
 ('i like fish', 1.0598181288715858)]

'can'

In [33]:
# Comlete the function on generating sequences. For this you will follow a greedy approach where you select the next word on the basis of highest bi-gram probability.
# Basically if the last word is w, you find the word x which has the highest bi-gram probability P(w,x)
# The function takes the starting word and the length of the sequence you want to generate
def generate(start_word, length = 2):
    output = [start_word]
    # You need to generate length-1 number of words so loop over that many times
    for i in range(0, length-1):
        # If last word in output(till now) is not in bigram_probabilities we will stop the generation
        if output[i] not in bigram_probabilities:
            break
        else:
          # Find the Bi-gram probabilities of all words with the last word in output(till now). The last word in output(till now) is basically output[i]
          # Then find the word which has the highest probability and add it to the output string
          # Your code here
          output.append(max(bigram_probabilities[output[i]], key=bigram_probabilities[output[i]].get))

    return output

In [34]:
# Use the generate function to generate a sequence of length 10 starting with the word "cats"
# Your Code here
generate('cats', 10)

['cats', 'are', 'the', 'first', 'time', 'in', 'the', 'first', 'time', 'in']

In [35]:
# Use the generate function to generate a sequence of length 10 starting with the word "we"
# Your Code here
generate('we', 10)

['we', 'can', 'be', 'a', 'commune', 'it', 'is', 'a', 'commune', 'it']

## Watermarking

In [36]:
# Two sentences
s1 = "pigeons known for their remarkable homing ability and adaptability to urban environments have been companions and messengers to humans for thousands of years ."
s2 = "with their iridescent feathers and distinctive cooing pigeons add a unique charm to cityscapes around the world ."

In [37]:
w = s1.split(" ") + s2.split(" ")
vocab = set(w)

In [40]:
import pickle

with open('data/red_list.data', 'rb') as f:
        red_list = pickle.load(f)

# red_list contains which are words are red/green as next words. If the value of red_list[w1][w2] is True that means w2 cannot follow w1 and it is a violation.
# If the value of red_list[w1][w2] is False that means w2 can follow w1 and it is not a violation.

In [41]:
red_list['with']['their']

True

In [42]:
red_list['and']['distinctive']

False

In [48]:
# Now you have the red list in red_list
# Write a function to check the number of violations
# The function takes the sentence s and the redlist as inputs
def check(s, redlist):
    # Get the list of words
    w = s.split(" ")
    # Count number of violations
    # Remember the red list contains true and flase
    # You need to check if the value corresponding to consecutive words in w are True in redlist. If so violations count will go up by 1.
    violations = 0
    for i in range(0, len(w)-1):
        violations += 1 if  redlist[w[i]][w[i+1]] is True else 0
    return violations

check(s1, red_list), check(s2, red_list)

(0, 10)

In [ ]:
# Calculate the probability of a human writing a sentence of the same length as s1 without ever violating the red list


In [ ]:
# Calculate the probability of a human writing a sentence of the same length as s2 without ever violating the red list
